In [ ]:
"""
Versão 1.1:
Melhoria no sistema de extração dos PDFs, sem usar LlamaParser, apenas bibliotecas de PDF. Também inclusa extração de tabelas
Modelo de Embedding atualizado para text-embedding-3-small
Divisão dos chunks ainda realizada em modelo de blocos de tamanho fixo
Com ajuda do GPT, tentamos implementar um esquema de busca inteligente dos documentos, citando nome específico e retornando contexto.
Número de resultados retornados com as queries foi aumentado 3->20 
"""

# Preparação dos Documentos

In [1]:
import pdfplumber
#import pytesseract
#import pdf2image
import pandas as pd
import tiktoken
import chromadb
import openai
import json
import os
import re
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings

In [2]:
CHUNK_SIZE = 1000
OFFSET = 200

tokenizer = tiktoken.get_encoding("cl100k_base")
openai_client = OpenAI(api_key= os.environ["OPENAI_API_KEY"])

# Conexão com o cliente do banco de dados ChromaDB
chromadb_path = "G:/Drives compartilhados/RISCO E COMPLIANCE/Relatórios de Risco/Risco/FIDCS/SCRIPTS_RISCO/Projeto IA/Base ChromaDB/"
chroma_client = chromadb.PersistentClient(path= chromadb_path)
collection    = chroma_client.get_or_create_collection(name= "Regulamentos_V1.1")

In [4]:
# Checa todo o conteúdo inserido no ChromaDB
collection.get()

{'ids': ['CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_0',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_1',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_2',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_3',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_4',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_5',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_6',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_7',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_8',
  'CONCRETO-

In [3]:
def count_tokens(text):
    return len(tokenizer.encode(text))

def split_document(document_text):
    documents = []
    for i in range(0, len(document_text), CHUNK_SIZE):
        start = i
        end = i + CHUNK_SIZE
        if start != 0:
            start = start - OFFSET
            end =  end - OFFSET
        documents.append(document_text[start: end])
    return documents

# Obtenção dos embeddings usando Langchain
def get_embedding_langchain(text):

    embedding = OpenAIEmbeddings(
        model= "text-embedding-3-small",
        chunk_size= CHUNK_SIZE
    )

    emb = embedding.embed_query(text)

    return emb

# Obtenção dos embeddings usando diretamente a API da OpenAI
def get_embedding_openai(text, client):

    emb = client.embeddings.create(
        input= text,
        model= "text-embedding-3-small"
    )

    return emb.data[0].embedding

# Extração de texto dos arquivos PDF
def extract_text_from_pdf(pdf_path):
    full_text = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            full_text.append(page.extract_text() or "")

    return "\n".join(full_text)

# Extração das tabelas dos arquivos PDF
# Tabelas serão extraídas diferentemente dos blocos de texto
def extract_tables_from_pdf(pdf_path):
    data_list = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            for table in tables:
                df = pd.DataFrame(table[1:], columns=table[0])  # Usa a primeira linha como cabeçalho
                data_list.append(df)

    return data_list

# Guarda os dados extraidos dos documentos no ChromaDB
def insert_chromadb(data, source, client):

    chunks = split_document(data)

    print(chunks)

    for idx, chunk in enumerate(chunks):

        embedding = get_embedding_openai(chunk, client)

        collection.add(
            ids=[f"{source}_chunk_{idx}"],
            documents=[chunk],
            metadatas=[{"source": source, "chunk_index": idx}],
            embeddings=[embedding]
        )

In [45]:
def run():
    print("Preparando Documentos...")
    data_path = 'G:/Drives compartilhados/GESTAO/_Operacional/Planilhas Gestão/Scripts/temp/temp_regulamentos/Teste 1.1'

    documents_names = [f for f in os.listdir(data_path) if f.endswith('.pdf')]
    documents_names_size = len(documents_names)

    for i, document_name in enumerate(documents_names):

        print(f"{i+1}/{documents_names_size}: {document_name}")

        doc_path = os.path.join(data_path, document_name)

        document_text = extract_text_from_pdf(os.path.join(data_path, document_name))
        if document_text.strip():
            insert_chromadb(document_text, document_name, openai_client)

        tables = extract_tables_from_pdf(doc_path)
        if tables:
            for idx, df in enumerate(tables):
                json_data = df.to_json(orient= "records")
                insert_chromadb(json_data, document_name, openai_client)

if __name__ == "__main__":
    run()
    pass

Preparando Documentos...
1/1: Regulamento Poupacred II.pdf
['REGULAMENTO DO\nFUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS POUPACRED II RESPONSABILIDADE\nLIMITADA\nCNPJ/MF Nº 57.377.181/0001-97\nVigente a partir de 24 janeiro de 2025\nSUMÁRIO\nPARTE GERAL ................................................................................................................................ 4\nCAPÍTULO I – DO FUNDO .......................................................................................................... 4\nCAPÍTULO II – DAS DEFINIÇÕES ................................................................................................ 4\nCAPÍTULO III - DO OBJETIVO DO FUNDO E DAS CLASSES DE COTAS ....................................... 8\nCAPÍTULO IV – DOS PRESTADORES DE SERVIÇOS ESSENCIAIS DO FUNDO ............................. 8\nCAPÍTULO V – DOS DEMAIS PRESTADORES DE SERVIÇOS .................................................... 13\nDO FUNDO ........................................

Insert of existing embedding ID: Regulamento Poupacred II.pdf_chunk_0
Add of existing embedding ID: Regulamento Poupacred II.pdf_chunk_0


In [50]:
# Checa todo o conteúdo inserido no ChromaDB
collection.get()

{'ids': ['CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_0',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_1',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_2',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_3',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_4',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_5',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_6',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_7',
  'CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA.pdf_chunk_8',
  'CONCRETO-

In [ ]:
def delete_existing_chunks(source):
    existing_chunks = collection.get()["ids"]

    for chunk_id in existing_chunks:
        if chunk_id.startswith(source):
            collection.delete(ids=[chunk_id])
            print(f"🗑️ Chunk {chunk_id} deletado.")

# Exemplo: Deletar apenas os chunks do documento "CONCRETO-CONSIGNADO"
#delete_existing_chunks("CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO")

# Implementação da RAG

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

def detect_document_mention(query, available_sources):
    for source in available_sources:
        if re.search(re.escape(source), query, re.IGNORECASE):  # Busca pelo nome do documento na pergunta
            return source
    return None  # Retorna None se nenhum documento for encontrado

# Função que pesquisa pelos documentos relevantes baseando-se no contexto da pergunta
def search_document(question, client):

    print("Pesquisando documentos relevantes...")

    query_embedding = get_embedding_openai(question, client)

    results = collection.query(
        query_embeddings= [query_embedding],
        n_results= 20
    )

    for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
        print(f"\nFonte: {meta['source']} - Tipo: {meta['type']}")
        print(doc[:1000])  

def ask_llm(query, client):

    stored_data = collection.get(include=["metadatas"])
    available_sources = {meta["source"] for meta in stored_data["metadatas"]}  # Obtém nomes únicos dos documentos

    # Verifica se o usuário mencionou um documento específico
    specific_document = detect_document_mention(query, available_sources)

    # Definição do filtro para busca no ChromaDB
    search_filter = {"source": specific_document} if specific_document else None  # Filtra apenas se um documento for citado

    # Busca informações no ChromaDB
    embedding = get_embedding_openai(query, client)

    results = collection.query(
        query_embeddings=[embedding],
        n_results=20,
        where= search_filter
    )

    # Organiza os resultados agrupando por 'source' (nome do documento)
    retrieved_docs = results["documents"][0]
    metadatas = results["metadatas"][0]

    grouped_docs = {}  # Dicionário para agrupar os chunks pelo nome do documento

    for doc, meta in zip(retrieved_docs, metadatas):
        source = meta["source"]  # Nome do documento original
        if source not in grouped_docs:
            grouped_docs[source] = []
        grouped_docs[source].append(doc)  # Adiciona o chunk ao documento correspondente

    # Criar um contexto consolidado por documento
    context = "\n\n".join([
        f"🔹 Documento: {source}\n" + "\n".join(chunks)
        for source, chunks in grouped_docs.items()
    ])

    modelo = ChatOpenAI(model = "gpt-4o-mini", temperature= 0, max_tokens= 4096)

    prompt = f"""
    Você é um assistente especializado em responder perguntas sobre vários documentos PDF armazenados em um banco de dados.
    Cada documento está dividido em múltiplos IDs, diferenciando-se apenas pelo número ao final.
    Documentos cujos IDs tenham nomes diferente não possuem relação alguma um com o outro.
    Quando for dado o nome de um fundo de investimento, forneça informação contida apenas naquele chunk. Não misture informação de diferentes documentos
    Considere os seguintes documentos: 
    {context}
    
    Caso não seja mencionado nenhum nome de fundo, responda considerando **todos** os documentos mencionados acima.
    """
    #Se não for encontrado nenhum documento semelhante ao nome dado, informe.
    messages=[
    SystemMessage(content= prompt),
    HumanMessage( content= query)
    ]

    answer = modelo.invoke(messages)

    return answer.content
    # return response["choices"][0]["message"]["content"]

# Aplicação do LLM

In [9]:
# question = """
# Quais são os gestores dos fundos Concreto e Concrédito II?
# """

question = """
Quem são as gestoras dos FIDC GVN?
"""

print(ask_llm(question, openai_client))

Não há informações sobre um fundo de investimento chamado "FIDC GVN" nos documentos mencionados. Os documentos disponíveis tratam de outros fundos, como o "CONCRETO-CONSIGNADO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS RESPONSABILIDADE LIMITADA", que é gerido pela CONCRETA GESTORA DE RECURSOS LTDA. e co-gestorado pela BLESS CAPITAL GESTORA DE RECURSOS LTDA., e o "FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS CONCRÉDITO II", que é gerido pela CULTINVEST ASSET MANAGEMENT LTDA. 

Se precisar de informações sobre esses ou outros fundos, por favor, me avise!


## Aplicação do LLM (ANTIGO)

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

modelo_antigo = ChatOpenAI(model = "gpt-4o-mini", temperature= 0, max_tokens= 4096)

prompt = """Você é um assistente de IA que responde as dúvidas dos usuários com bases nos documentos abaixo.
Os documentos abaixo apresentam as fontes atualizadas e devem ser consideradas como verdade. Sempre que uma
pergunta oferecer o nome de um FIDC, você deve procurar pela melhor correspondência deste nome na lista de documentos.
Cite a fonte quando fornecer a informação.
Documentos:
{documents}
""" #Caso o nome citado na pergunta não exista na lista de documentos, responda que ele não foi encontrado.

prompt = prompt.format(documents= documents_str)

messages=[
  SystemMessage(content= prompt),
  HumanMessage(content= question)
]

In [10]:
answer = modelo_antigo.invoke(messages)

In [11]:
answer.content

"Com base nos documentos disponíveis, não encontrei menções aos termos 'Saque aniversário', 'FGTS', 'INSS', 'Consignado' ou 'Saque-Aniversário FGTS'. Portanto, não há documentos que incluam qualquer um desses termos."